1. 加载数据
2. 切片
3. 删除无信息（安静）片段
4. 精简数据（how？）
5. fft
6. t-SNE
7. knn cluster
8. write into static dict

In [ ]:
!pip install SpeechRecognition
!pip install soundfile
## !pip install cuml

In [1]:
import os
import shutil
from tqdm import tqdm
import random
import math
from glob import glob

import soundfile as sf            # To read .flac files.   
import speech_recognition as sr   # pip install SpeechRecognition.
from sklearn.manifold import TSNE
# import cuml
# import cudf
# from cuml.manifold import TSNE as cudaTSNE
import pickle as pkl

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
# static parameters
durationCheck = 10.      # Only consider files with 10 or more seconds of audio.
deltaT        = 0.2      # Audio frame size is 0.2 seconds.
noisy         = 0.1      # This sets the limit for static, i.e. pauses in speech.
lim1 = 10; lim2 = 410    # Lower and upper frequencies. 
                         # For the above parameters and 16 kHz sampling, this range is about 50 - 2000 Hz.  

fft_numFeatures = lim2-lim1

In [3]:
# get the current directory
proj_root_dir = os.path.abspath(os.path.join(os.getcwd(),".."))
proj_root_dir

train_folder_dir = os.path.join(proj_root_dir, 'data', 'train')
state_dict_folder_dir = os.path.join(proj_root_dir, 'state_dict')

In [5]:
# fft calculation

# Read data from a folder into a list.
def getSpeakerData(speaker_folder_name):
  flac_file_list = glob(os.path.join(train_folder_dir, speaker_folder_name, '*.flac'))
  flac_dir_list = []
  for flac_file_dir in flac_file_list:
    data,samplerate = sf.read(flac_file_dir)  
    duration = len(data)*1./samplerate
    if duration >= durationCheck: 
      flac_dir_list.append(flac_file_dir)

  chunksF = []

  for flac_dir in flac_dir_list:
    data,samplerate = sf.read(flac_dir)  
    duration = len(data)*1./samplerate

    # Divide audio data into frames, or chunks. 
    numChunks = int(duration/deltaT)
    sizeChunk = int(len(data)/numChunks)
    for lp in range(0,numChunks):    
      chunk = data[lp*sizeChunk:(lp+1)*sizeChunk]      # get a chunk of speech.     
      chunksF.append(np.abs(np.fft.rfft(chunk))[lim1:lim2])  # take the FFT.
      # shape of chunksF: slice_num * 400

    # Delete quiet parts of speech, i.e. pauses.
    # Most of the power is in the bottom 50% of frequencies.
    mu = np.mean([np.mean(chunksF[i][:fft_numFeatures//2]) for i in range(0,len(chunksF))])
    speaker_fft = []
    for chunkF in chunksF:
      if np.mean(chunkF[:fft_numFeatures//2]) > noisy*mu:
        speaker_fft.append(chunkF)
        # shape of speaker_fft: slice_w_information_num * 400
    
  return speaker_fft

# Return data for all speakers.
def getDataSpeakers(division):
  dataSpeakers = []
  for speaker in tqdm(sorted(os.listdir(train_folder_dir))):
    # print ("Getting data for speaker: "+speaker)
    dataSpeakers.append(getSpeakerData(speaker))
    # shape pf dataSpeakers: speaker_num * slice_w_info_num * 400

  N = np.sum([np.shape(s)[0] for s in dataSpeakers])
  tX = np.mat(np.zeros((N,fft_numFeatures)))
  tY = []
  speakerIndices = [0]    # Index corresponding to start of speaker 'n'
  
  ctr = 0; lp = 0
  for dataSpeaker in dataSpeakers:
    for j in range(0,len(dataSpeaker)):
      for k in range(0,fft_numFeatures):
        tX[ctr,k] = dataSpeaker[j][k]
      tY.append(lp)
      ctr += 1  
    speakerIndices.append(ctr)
    lp += 1  
          
  return tX,tY,speakerIndices

In [6]:
# overall fft information
trnX,trnY,trnIdx = getDataSpeakers('train')
print(np.shape(trnX), np.shape(trnY))
trnRows = np.shape(trnX)[0]
print(trnIdx)   # Start location of speaker 'i'.

100%|██████████| 250/250 [13:30<00:00,  3.24s/it]


(890832, 400) (890832,)
[0, 3650, 7201, 10359, 14048, 17013, 20314, 24094, 27916, 31546, 34856, 38789, 42464, 46112, 47149, 51585, 54519, 58566, 62226, 66093, 68765, 72126, 75933, 79924, 84245, 88204, 91834, 95642, 99009, 102640, 105770, 108934, 112755, 116612, 120566, 124878, 128706, 131840, 135547, 138740, 141234, 145418, 148843, 152095, 155520, 159346, 163149, 166726, 170369, 174148, 178114, 181995, 185619, 189572, 193905, 197345, 201424, 205386, 208756, 212561, 217408, 220975, 224581, 227512, 231245, 234814, 238309, 240510, 244101, 247141, 250840, 254531, 257907, 261665, 264651, 268076, 272187, 276252, 280357, 283882, 287849, 291296, 293157, 296767, 299990, 301603, 304875, 308798, 311652, 315156, 319373, 323210, 327191, 330950, 334811, 338223, 341625, 345355, 349213, 352937, 356612, 359509, 363569, 367398, 371042, 374740, 378333, 381844, 385455, 389616, 393245, 397141, 400865, 404708, 408332, 412095, 415234, 418707, 422654, 426583, 430437, 434202, 437474, 440874, 444794, 448730, 45

In [7]:
try:
    os.mkdir(os.path.join(state_dict_folder_dir, 'fft_result_wo_simplify'))
except OSError:
    pass

with open(os.path.join(state_dict_folder_dir, 'fft_result_wo_simplify', 'trnX.pkl'), 'wb') as f:
    pkl.dump(trnX, f)
with open(os.path.join(state_dict_folder_dir, 'fft_result_wo_simplify', 'trnY.pkl'), 'wb') as f:
    pkl.dump(trnY, f)
with open(os.path.join(state_dict_folder_dir, 'fft_result_wo_simplify', 'trnIdx.pkl'), 'wb') as f:
    pkl.dump(trnIdx, f)

In [4]:
# t-SNE dimentional reduction

with open('/data1/home/xiruiling/course/AdvanceArtificialIntelligence/AAI_Proj/state_dict/fft_result_wo_simplify/trnX.pkl', 'rb') as f:
    trnX = pkl.load(f)
print("The original dimension: ", np.array(trnX).shape)
x_tsne = TSNE(n_components=2,random_state=0).fit_transform(trnX)
# x_tsne = cudaTSNE(n_components=2, perplexity=50, learning_rate=20).fit_transform(trnX)
x_tsne
# print("after t-SNE: ", x_tsne.shape)

KeyboardInterrupt: 

In [ ]:
# visualize the t-SNE result
x_min, x_max = x_tsne.min(0), x_tsne.max(0)
X_norm = (x_tsne - x_min) / (x_max - x_min)  # 归一化
plt.figure(figsize=(8, 8))
for i in range(X_norm.shape[0]):
    plt.text(X_norm[i, 0], X_norm[i, 1], str(y[i]), color=plt.cm.Set1(y[i]), 
             fontdict={'weight': 'bold', 'size': 9})
plt.xticks([])
plt.yticks([])
plt.show()